In [4]:
from feature_reader import process_files_randomly, preprocess_data, read_and_process_polars
feature, target = process_files_randomly('processed_data', 100)
#feature, target = preprocess_data(feature, target, feature.columns, target.columns)


In [2]:
from linear_model import train_and_evaluate
from linear_model import preprocess_data as lm_preprocess_data
from sklearn.model_selection import train_test_split

X, y = lm_preprocess_data(feature, target)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the models with L1 and L2 regularization
ridge_model, lasso_model = train_and_evaluate(X_train, X_test, y_train, y_test, alpha=1.0)

# Print out the coefficients of the Ridge regression model
ridge_coefficients = ridge_model.coef_
ridge_coefficients


/Users/ekko/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
target.columns

['token0_value_5s',
 'token0_value_10s',
 'token0_value_30s',
 'token0_value_60s',
 'token0_value_300s',
 'token0_value_600s',
 'token0_value_1800s',
 'token0_value_3600s']

In [ ]:
feature.columns

['open_token0',
 'current_token0',
 'token0_value_5s',
 'token0_value_10s',
 'token0_value_30s',
 'token0_value_60s',
 'token0_value_120s',
 'token0_value_300s',
 'token0_value_600s',
 'token0_value_1200s',
 'token0_value_1800s',
 'token0_value_3600s',
 'inflow_5s',
 'outflow_5s',
 'inflow_10s',
 'outflow_10s',
 'inflow_30s',
 'outflow_30s',
 'inflow_60s',
 'outflow_60s',
 'inflow_120s',
 'outflow_120s',
 'inflow_300s',
 'outflow_300s',
 'inflow_600s',
 'outflow_600s',
 'inflow_1200s',
 'outflow_1200s',
 'inflow_1800s',
 'outflow_1800s',
 'inflow_3600s',
 'outflow_3600s',
 'negative_holdings',
 'num_addresses',
 'max_address_holding',
 'top_5_address_holding',
 'top_10_address_holding',
 'holding_entropy',
 'elapsed_time_since_open',
 'time_of_day_morning',
 'time_of_day_afternoon',
 'time_of_day_evening',
 'time_of_day_night']

In [ ]:
feature

open_time,open_token0,open_token1,open_liquidity,current_slot,current_token0,current_token1,current_liquidity_ratio,current_to_open_token0_ratio,time_of_day,token0_value_15slots,token0_relative_value_15slots,token0_diff_value_15slots,token0_relative_diff_value_15slots,token0_value_30slots,token0_relative_value_30slots,token0_value_60slots,token0_relative_value_60slots,token0_diff_value_30slots,token0_relative_diff_value_30slots,token0_value_120slots,token0_relative_value_120slots,token0_diff_value_60slots,token0_relative_diff_value_60slots,token0_value_240slots,token0_relative_value_240slots,token0_diff_value_120slots,token0_relative_diff_value_120slots,token0_value_480slots,token0_relative_value_480slots,token0_diff_value_240slots,token0_relative_diff_value_240slots,token0_value_960slots,token0_relative_value_960slots,token0_diff_value_480slots,token0_relative_diff_value_480slots,token0_value_1920slots,…,outflow_7680slots,negative_holdings,num_addresses,max_address_holding,top_5_address_holding,top_10_address_holding,holding_entropy,top_5_address_holding_diff_15slots,top_10_address_holding_diff_15slots,max_address_holding_diff_15slots,top_5_address_holding_diff_30slots,top_10_address_holding_diff_30slots,max_address_holding_diff_30slots,top_5_address_holding_diff_60slots,top_10_address_holding_diff_60slots,max_address_holding_diff_60slots,top_5_address_holding_diff_120slots,top_10_address_holding_diff_120slots,max_address_holding_diff_120slots,top_5_address_holding_diff_240slots,top_10_address_holding_diff_240slots,max_address_holding_diff_240slots,top_5_address_holding_diff_480slots,top_10_address_holding_diff_480slots,max_address_holding_diff_480slots,top_5_address_holding_diff_960slots,top_10_address_holding_diff_960slots,max_address_holding_diff_960slots,top_5_address_holding_diff_1920slots,top_10_address_holding_diff_1920slots,max_address_holding_diff_1920slots,top_5_address_holding_diff_3840slots,top_10_address_holding_diff_3840slots,max_address_holding_diff_3840slots,top_5_address_holding_diff_7680slots,top_10_address_holding_diff_7680slots,max_address_holding_diff_7680slots
datetime[μs],i64,i64,f64,i64,i64,i64,f64,f64,str,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,…,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
2024-06-16 17:35:29.951635,85000000000,1000000000000000,8.5000e25,271588081,87417530702,972873235658768,1.000543,1.028442,"""afternoon""",87903058227,1.034154,0,0.0,87903058227,1.034154,87903058227,1.034154,0,0.0,86683216533,1.019803,-1219841694,-0.014351,85000000000,1.0,-1683216533,-0.019803,85000000000,1.0,0,0.0,85000000000,1.0,0,0.0,85000000000,…,-117617486560,-92212964,39,0.219146,0.81582,0.970537,2.13762,-0.003647,-0.006626,-0.088154,-0.018452,-0.011427,-0.093706,-0.019474,-0.014893,0.002588,-0.169335,-0.027694,-0.180107,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0
2024-06-16 17:35:29.951635,85000000000,1000000000000000,8.5000e25,271588384,90451357490,940889515564748,1.001232,1.064134,"""afternoon""",87903058227,1.034154,0,0.0,87903058227,1.034154,87903058227,1.034154,0,0.0,87903058227,1.034154,0,0.0,87903058227,1.034154,0,0.0,85000000000,1.0,-2903058227,-0.034154,85000000000,1.0,0,0.0,85000000000,…,-117617486560,-253660532,46,0.306747,0.885186,0.958507,1.933659,-0.000998,-0.00036,0.002665,0.001908,-0.000739,-0.061285,0.122533,0.033909,-0.100877,0.073115,0.005571,0.089433,0.025526,-0.010687,-0.059966,-0.105116,-0.039652,-0.144689,0,0,0,0,0,0,0,0,0,0,0,0
2024-06-16 17:35:29.951635,85000000000,1000000000000000,8.5000e25,271587899,87879270959,967316198585332,1.000083,1.033874,"""afternoon""",85000000000,1.0,0,0.0,85000000000,1.0,85000000000,1.0,0,0.0,85000000000,1.0,0,0.0,85000000000,1.0,0,0.0,85000000000,1.0,0,0.0,85000000000,1.0,0,0.0,85000000000,…,-117617486560,-1724,45,0.451503,0.990448,0.998263,1.265438,-0.00074,0.000507,-0.205324,0.0268

In [3]:
feature

open_token0,open_token1,open_liquidity,current_slot,current_token0,current_token1,current_liquidity_ratio,current_to_open_token0_ratio,slot_elapse,token0_value_15slots,token0_relative_value_15slots,token0_diff_value_15slots,token0_relative_diff_value_15slots,token0_value_30slots,token0_relative_value_30slots,token0_value_60slots,token0_relative_value_60slots,token0_diff_value_30slots,token0_relative_diff_value_30slots,token0_value_120slots,token0_relative_value_120slots,token0_diff_value_60slots,token0_relative_diff_value_60slots,token0_value_240slots,token0_relative_value_240slots,token0_diff_value_120slots,token0_relative_diff_value_120slots,token0_value_480slots,token0_relative_value_480slots,token0_diff_value_240slots,token0_relative_diff_value_240slots,token0_value_960slots,token0_relative_value_960slots,token0_diff_value_480slots,token0_relative_diff_value_480slots,token0_value_1920slots,token0_relative_value_1920slots,…,top_5_address_holding,top_10_address_holding,holding_entropy,top_5_address_holding_diff_15slots,top_10_address_holding_diff_15slots,max_address_holding_diff_15slots,top_5_address_holding_diff_30slots,top_10_address_holding_diff_30slots,max_address_holding_diff_30slots,top_5_address_holding_diff_60slots,top_10_address_holding_diff_60slots,max_address_holding_diff_60slots,top_5_address_holding_diff_120slots,top_10_address_holding_diff_120slots,max_address_holding_diff_120slots,top_5_address_holding_diff_240slots,top_10_address_holding_diff_240slots,max_address_holding_diff_240slots,top_5_address_holding_diff_480slots,top_10_address_holding_diff_480slots,max_address_holding_diff_480slots,top_5_address_holding_diff_960slots,top_10_address_holding_diff_960slots,max_address_holding_diff_960slots,top_5_address_holding_diff_1920slots,top_10_address_holding_diff_1920slots,max_address_holding_diff_1920slots,top_5_address_holding_diff_3840slots,top_10_address_holding_diff_3840slots,max_address_holding_diff_3840slots,top_5_address_holding_diff_7680slots,top_10_address_holding_diff_7680slots,max_address_holding_diff_7680slots,time_of_day_morning,time_of_day_afternoon,time_of_day_evening,time_of_day_night
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i8,i8,i8,i8
NaN,NaN,0.995086,1.617745,0.304607,-0.09728,0.107967,0.304607,1.617745,1.132621,1.132621,0.182405,0.182405,1.145057,1.145057,1.361896,1.361896,0.428215,0.428215,1.520525,1.520525,0.307691,0.307691,0.971699,0.971699,-0.961944,-0.961944,1.832518,1.832518,0.640658,0.640658,NaN,NaN,-1.832518,-1.832518,NaN,NaN,…,-0.64987,-1.304228,0.801985,0.896238,0.835523,-1.07105,2.052293,1.894523,0.565938,0.309993,-0.369256,-0.130362,-0.076565,-1.102121,-0.653941,0.652188,-0.780403,1.326728,0.422991,-1.554901,1.388489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0
NaN,NaN,0.995086,-0.051774,0.190956,0.004155,0.098782,0.190956,-0.051774,-0.114676,-0.114676,0.182405,0.182405,-0.035374,-0.035374,0.376246,0.376246,0.642903,0.642903,0.568147,0.568147,0.307691,0.307691,-0.337383,-0.337383,-1.178642,-1.178642,-0.505236,-0.505236,-0.09799,-0.09799,NaN,NaN,0.505236,0.505236,NaN,NaN,…,-0.88436,0.174443,0.713594,0.065579,0.627346,1.345953,0.127513,0.234147,1.155846,0.118664,0.081716,0.610671,-1.448767,-0.518968,0.42363,-2.588429,-0.517393,-3.93376,0.434691,0.522008,0.086754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0
NaN,NaN,0.995086,0.936757,0.315224,-0.111516,0.103801,0.315224,0.936757,1.132621,1.132621,-2.76094,-2.76094,-0.035374,-0.035374,0.239371,0.239371,0.428215,0.428215,0.435892,0.435892,0.307691,0.307691,0.971699,0.971699,0.534886,0.534886,-0.505236,-0.505236,-1.579725,-1.579725,NaN,NaN,0.505236,0.505236,NaN,NaN,…,-0.238421,-0.415945,0.414775,1.619023,1.747798,-0.861732,2.629571,2.150299,-0.287254,2.67592,2.855512,1.21413,0.623

In [5]:
feature

open_time,open_token0,open_token1,open_liquidity,current_slot,current_token0,current_token1,current_liquidity_ratio,current_to_open_token0_ratio,slot_elapse,token0_value_15slots,token0_relative_value_15slots,token0_diff_value_15slots,token0_relative_diff_value_15slots,token0_value_30slots,token0_relative_value_30slots,token0_value_60slots,token0_relative_value_60slots,token0_diff_value_30slots,token0_relative_diff_value_30slots,token0_value_120slots,token0_relative_value_120slots,token0_diff_value_60slots,token0_relative_diff_value_60slots,token0_value_240slots,token0_relative_value_240slots,token0_diff_value_120slots,token0_relative_diff_value_120slots,token0_value_480slots,token0_relative_value_480slots,token0_diff_value_240slots,token0_relative_diff_value_240slots,token0_value_960slots,token0_relative_value_960slots,token0_diff_value_480slots,token0_relative_diff_value_480slots,token0_value_1920slots,…,negative_holdings,num_addresses,max_address_holding,top_5_address_holding,top_10_address_holding,holding_entropy,top_5_address_holding_diff_15slots,top_10_address_holding_diff_15slots,max_address_holding_diff_15slots,top_5_address_holding_diff_30slots,top_10_address_holding_diff_30slots,max_address_holding_diff_30slots,top_5_address_holding_diff_60slots,top_10_address_holding_diff_60slots,max_address_holding_diff_60slots,top_5_address_holding_diff_120slots,top_10_address_holding_diff_120slots,max_address_holding_diff_120slots,top_5_address_holding_diff_240slots,top_10_address_holding_diff_240slots,max_address_holding_diff_240slots,top_5_address_holding_diff_480slots,top_10_address_holding_diff_480slots,max_address_holding_diff_480slots,top_5_address_holding_diff_960slots,top_10_address_holding_diff_960slots,max_address_holding_diff_960slots,top_5_address_holding_diff_1920slots,top_10_address_holding_diff_1920slots,max_address_holding_diff_1920slots,top_5_address_holding_diff_3840slots,top_10_address_holding_diff_3840slots,max_address_holding_diff_3840slots,top_5_address_holding_diff_7680slots,top_10_address_holding_diff_7680slots,max_address_holding_diff_7680slots,time_of_day
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
1.7186e15,8.5000e10,1.0000e15,8.5000e25,2.715885e8,8.9775e10,9.4826e14,1.001523,1.056173,0.0,8.8379e10,1.039749,0.0,0.0,8.8379e10,1.039749,8.8379e10,1.039749,0.0,0.0,8.8379e10,1.039749,0.0,0.0,8.6977e10,1.023261,-1.4015e9,-0.016488,8.7148e10,1.025271,1.70887186e8,0.00201,8.5000e10,1.0,-2.1481e9,-0.025271,8.5000e10,…,-2.5724e8,51.0,0.327904,0.834664,0.922629,2.103772,0.028205,0.008272,-0.108378,0.081798,0.024633,0.043488,0.006248,-0.012411,-0.054442,-0.033346,-0.031287,-0.088266,0.022594,-0.030308,0.110589,-0.00063,-0.062802,0.111346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""afternoon"""
1.7186e15,8.5000e10,1.0000e15,8.5000e25,2.7158812e8,8.8776e10,9.5805e14,1.000613,1.044421,-380.0,8.6977e10,1.023261,0.0,0.0,8.6977e10,1.023261,8.7148e10,1.025271,1.70887186e8,0.00201,8.7148e10,1.025271,0.0,0.0,8.5544e10,1.006396,-1.6044e9,-0.018875,8.5000e10,1.0,-5.4370e8,-0.006396,8.5000e10,1.0,0.0,0.0,8.5000e10,…,-9.2598366e7,40.0,0.336054,0.814285,0.973472,2.056071,-0.002364,0.00564,0.102448,-0.004799,-0.000948,0.116032,-0.005181,-0.003691,0.028754,-0.124537,-0.02088,0.022824,-0.177574,-0.024607,-0.321218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""afternoon"""
1.7186e15,8.5000e10,1.0000e15,8.5000e25,2.71588345e8,8.9868e10,9.4688e14,1.00111,1.057271,-155.0,8.8379e10,1.039749,-1.4015e9,-0.016488,8.6977e10,1.023261,8.6977e10,1.023261,0.0,0.0,8.6977e10,1.023261,0.0,0.0,8.6977e10,1.023261,0.0,0.0,8.5000e10,1.0,-1.9772e9,-0.023261,8.5000e10,1.0,0.0,0.0,8.5000e10,…,-1.8697e8,46.0,0.346191,0.870422,0.953172,1.89481,0.054804,0.019806,-0.09012,0.10777,0.0285